<a href="https://colab.research.google.com/github/Bluedata-Consulting/GAAPB02-training/blob/main/SLEFLEARNING_Prompt_Engineering_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Techniques



This guide provides an introduction to prompt engineering techniques for Azure OpenAI services, focusing on how to craft effective prompts to elicit desired responses from large language models (LLMs).

### Core Concepts

**What is Prompt Engineering?**

Prompt engineering is the art and science of designing effective prompts to guide LLMs toward generating the desired output. It's about understanding how LLMs process information and using that knowledge to create prompts that are clear, specific, and unambiguous.

**Why is Prompt Engineering Important?**

* **Accuracy:** Well-crafted prompts significantly improve the accuracy and relevance of the generated responses.
* **Control:**  Prompt engineering gives you control over the style, format, and content of the output.
* **Efficiency:** Effective prompts can minimize the number of iterations needed to achieve the desired results, saving time and resources.
* **Customization:** Prompt engineering allows you to tailor the LLM's behavior to specific tasks and domains.


### Key Techniques and Examples


**1.  Clear and Specific Instructions:**

* **Bad Prompt:**  "Tell me about dogs."  (Too vague, could result in a wide range of responses.)
* **Good Prompt:** "Write a 200-word essay comparing and contrasting the characteristics of Golden Retrievers and German Shepherds." (Specific, sets length and topic.)


**2.  Few-Shot Learning:**

Provide examples of input-output pairs to guide the model.

* **Example:**  


Make Sure, Below Environment Variables are created already:


```
AZURE_OPENAI_API_KEY
AZURE_OPENAI_ENDPOINT

```

In [ ]:
import os
#os.environ['AZURE_OPENAI_ENDPOINT'] = ""
#os.environ['AZURE_OPENAI_API_KEY'] = ""

In [1]:
from openai import AzureOpenAI
client = AzureOpenAI(api_version="2024-12-01-preview")

In [4]:
# creating a function to get outcome
def generate_response(prompt,model="myllm",temperature=0):
  messages = [{"role":"user","content":prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature=temperature
  )
  return response.choices[0].message.content


## Zero Shot Prompting

In [5]:
# zero shot prompting
prompt = f"""
Create a conversation story between child and grandparent.
"""
response = generate_response(prompt)
print(response)

**Title: The Magic of Memories**

**Setting:** A cozy living room filled with family photos, a warm fireplace, and the smell of freshly baked cookies.

**Characters:**
- **Lily:** A curious 8-year-old girl.
- **Grandpa Joe:** A wise and gentle grandfather with a twinkle in his eye.

---

**Lily:** (snuggling into the big armchair) Grandpa, can you tell me a story? A real one, about when you were a kid?

**Grandpa Joe:** (chuckling) Oh, I have plenty of those! Let me think… Ah! How about the time I built a treehouse with my friends?

**Lily:** A treehouse? That sounds amazing! How did you do it?

**Grandpa Joe:** Well, we didn’t have fancy tools like you do today. We used old wooden planks we found and some rusty nails. We climbed up into the big oak tree in my backyard. It felt like we were building a castle in the sky!

**Lily:** (eyes wide) Did you have a secret password to get in?

**Grandpa Joe:** (grinning) Of course! We decided that only the bravest adventurers could enter. Our p

## Few Shot Prompting

Here, the model is given a few examples (shots) to guide its response. By providing context or previous instances, the model can better understand and generate the desired output. For example, showing a model several examples of translated sentences before asking it to translate a new one.


In [6]:
# few shot prompting
prompt = f"""
your task is to answer in a consistent sytle:

<child>: Teach me about patience.
<grandma>: Patience is like waiting for whole day to see the moon in the evening and then sleeping after having a look at it.

<child>: Teach me about resilience.
"""
response = generate_response(prompt)
print(response)

<grandma>: Resilience is like a tree that bends in the wind but doesn’t break. It stands tall again after the storm passes, ready to grow even stronger.


#### Few shot prompting with role setup

In [8]:
result = client.chat.completions.create(
    model='myllm',  # e.g. gpt-35-instant
    max_tokens=200,
    temperature=0.9,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "That was an awesome experience"},
        {"role": "assistant", "content": "positive"},
        {"role": "user", "content": "I won't do that again"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "That was not worth my time"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "You can't miss this"}
    ],
)
print(result.choices[0].message.content)

positive


## Chain of Thought Prompting

In [9]:
# Without Chain of Thought Prompting
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

To find out how many pens Marry has now, we can break it down step by step:

1. Marry started with 5 pens.
2. She gave away 3 pens, so now she has:  
   \( 5 - 3 = 2 \) pens.
3. She then bought 2 boxes of pens, with 3 pens in each box. So, the total number of pens she bought is:  
   \( 2 \times 3 = 6 \) pens.
4. Now, we add the pens she has left to the pens she bought:  
   \( 2 + 6 = 8 \) pens.

So, Marry has 8 pens now.


In [10]:
# with Chain of Thought prompting
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: Johan had 5 apples, 4 boxes with 5 apples each = 5 + 4*5 = 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Marry had 5 pens, she gave away 3 pens, so she has:

5 - 3 = 2 pens left.

Then, she bought 2 boxes of pens with 3 pens in each box, which means she bought:

2 * 3 = 6 pens.

Now, we add the pens she has left to the pens she bought:

2 + 6 = 8 pens.

So, Marry has 8 pens now.


In [11]:
# with Chain of Thought prompting
prompt = f"""

Think step by step to answer below question, do not come to conclusion without following process.
Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Let's break down the problem step by step:

1. **Initial Number of Pens**: Marry starts with 5 pens.

2. **Pens Given Away**: She gives 3 pens to her friend. 
   - To find out how many pens she has left after giving away the pens, we subtract the number of pens given away from the initial number:
   \[
   5 - 3 = 2 \text{ pens}
   \]

3. **Buying New Pens**: Marry then buys 2 boxes of pens, with each box containing 3 pens.
   - First, we calculate the total number of pens in the boxes:
   \[
   2 \text{ boxes} \times 3 \text{ pens per box} = 6 \text{ pens}
   \]

4. **Total Number of Pens Now**: To find out how many pens Marry has now, we add the pens she has left after giving some away to the new pens she bought:
   \[
   2 \text{ pens} + 6 \text{ pens} = 8 \text{ pens}
   \]

So, after following all the steps, Marry has **8 pens** now.


## Tree of Thoughts (ToT)

Tree of Thoughts (ToT) prompting is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem-solving with language models.


How does it work?

- ToT prompting breaks problems down into smaller parts, similar to CoT prompting, but goes further by combining this with the ability to explore multiple solution paths in parallel, forming a tree instead of a single chain. Each thought is generated or solved independently and passed to the next step, allowing the model to self-evaluate and decide whether to continue with that path or choose another.

In [12]:

prompt = """
Imagine 5 different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on science and the laws of physics
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought
Each expert will assign a likelihood of their current assertion being correct
Continue until the experts agree on the single most likely location
The question is.
1. Carlos is at the swimming pool.
2. He walks to the locker room, carrying a towel.
3. He puts his watch in the towel and carries the towel tightly to a lounger at the poolside.
4. At the lounger he opens and vigorously shakes the towel, then walks to the snack bar.
5. He leaves the towel at the snack bar, then walks to the diving board.
6. Later Carlos realises he has has lost his watch. Where is the single most likely location of the watch?
"""


ans=generate_response(prompt)

print(ans)

### Expert 1: The Physical Movement Expert
**Step 1:** Carlos starts at the swimming pool and walks to the locker room carrying a towel. The first thing to consider is that he puts his watch in the towel before heading to the lounger. This means the watch is initially in the towel when he leaves the locker room. 

**Critique:** This step is solid as it establishes the initial condition of the watch being in the towel. However, I need to consider what happens to the towel after he shakes it at the lounger.

**Likelihood of correctness:** 90%

---

### Expert 2: The Towel Dynamics Expert
**Step 2:** After Carlos puts his watch in the towel and shakes it at the lounger, there is a possibility that the watch could fall out during the vigorous shaking. The towel is not a sealed container, so the watch could easily slip out if it was not held tightly.

**Critique:** This is a valid point. The action of shaking the towel introduces a risk of the watch falling out. However, we need to consider

## Chain-of-Verification Prompting

The Chain-of-Verification (CoVe) prompt engineering method aims to reduce hallucinations through a verification loop.

In [13]:
Question="Name some athletes who were born in United states"

prompt=f'''Here is the question: {Question}.

First, generate a response.

Then, create and answer verification questions based on this response to check for accuracy. Think it through and make sure you are extremely accurate based on the question asked.

After answering each verification question, consider these answers and revise the initial response to formulate a final, verified answer. Ensure the final response reflects the accuracy and findings from the verification process.'''



ans=generate_response(prompt)

print(ans)


**Initial Response:**
Some athletes who were born in the United States include:
1. Michael Jordan (Basketball)
2. Serena Williams (Tennis)
3. Tom Brady (American Football)
4. Simone Biles (Gymnastics)
5. LeBron James (Basketball)

**Verification Questions:**
1. **What sport is Michael Jordan known for?**
   - Answer: Basketball
   - Verification: Michael Jordan is widely recognized as one of the greatest basketball players of all time, having played primarily for the Chicago Bulls in the NBA.

2. **Is Serena Williams a professional athlete? If so, in what sport?**
   - Answer: Yes, she is a professional athlete in tennis.
   - Verification: Serena Williams is a highly successful tennis player, known for her numerous Grand Slam titles and contributions to the sport.

3. **What position does Tom Brady play in American Football?**
   - Answer: Quarterback
   - Verification: Tom Brady is known for his role as a quarterback, having played for the New England Patriots and the Tampa Bay Bucca

# Model Limitations: Hallucinations

In [14]:
# asking for a specific format
text = """
Washing clothes without washing machines is easy!
First you need to get some warm water. Mix the warm water with some detergent. Second you can put the dirty clothes in it and let it stay wet for some time.
During this period the stains will get loose and mixed with water.
thereafter you can take clothes out, rub the dirt/stains with a soft brush to remove the stains if any.
get the clothes mixed with water to get the detergent out of it. and then put it for drying on a rope.
Enjoy the clean clothes.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks. Rewrite those into a set of instructions in the following format:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```
"""
response = generate_response(prompt)
print(response)


Step 1: Gather warm water and mix it with some detergent.  
Step 2: Place the dirty clothes into the mixture and let them soak for a while.  
Step 3: Allow the stains to loosen and mix with the water during the soaking period.  
Step 4: Remove the clothes from the water and use a soft brush to rub off any remaining dirt or stains.  
Step 5: Rinse the clothes in clean water to remove the detergent.  
Step 6: Hang the clothes on a rope to dry.  
Step 7: Enjoy your clean clothes!


In [15]:
# asking for a specific format
text = """
THe morning today is amazing, the sun is looking bright in between clouds, the weather is pleasent and the wind is mindblowing. on a whole i am loving the day today.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks. Rewrite those into a set of instructions in the following format:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```
"""
response = generate_response(prompt)
print(response)


Step 1: Observe the morning weather and take note of the brightness of the sun.  
Step 2: Look for clouds in the sky and appreciate the contrast with the sunlight.  
Step 3: Enjoy the pleasant weather and take a moment to feel the wind.  
Step 4: Reflect on your overall feelings about the day and embrace the positivity.


In [16]:
# asking for a specific format
text = """
Washing clothes without washing machines is easy!
First you need to get some warm water. Mix the warm water with some detergent. Second you can put the dirty clothes in it and let it stay wet for some time.
During this period the stains will get loose and mixed with water.
thereafter you can take clothes out, rub the dirt/stains with a soft brush to remove the stains if any.
get the clothes mixed with water to get the detergent out of it. and then put it for drying on a rope.
Enjoy the clean clothes.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks.
Rewrite those into a set of instructions in the following format only if contains a sequence of instructions:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```

if the text does not contain a sequence of instructions, then simply write 'No steps provided'
"""
response = generate_response(prompt)
print(response)


Step 1: Get some warm water.  
Step 2: Mix the warm water with some detergent.  
Step 3: Put the dirty clothes in the mixture and let them stay wet for some time.  
Step 4: Take the clothes out and rub the dirt/stains with a soft brush to remove any stains.  
Step 5: Rinse the clothes with water to get the detergent out.  
Step 6: Hang the clothes on a rope for drying.  


In [17]:
# asking for a specific format
text = """
THe morning today is amazing, the sun is looking bright in between clouds, the weather is pleasent and the wind is mindblowing. on a whole i am loving the day today.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks.
Rewrite those into a set of instructions in the following format only if contains a sequence of instructions:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```

if the text does not contain a sequence of instructions, then simply write 'No steps provided'
"""
response = generate_response(prompt)
print(response)


No steps provided


In [18]:
prompt = f"""
Tell me about Ergonamic chair procuced and sold by Tredence.
"""

generate_response(prompt)

"As of my last knowledge update in October 2023, Tredence is primarily known as a data science and analytics company, focusing on providing insights and solutions to businesses across various industries. They specialize in areas such as AI, machine learning, and data engineering, rather than manufacturing or selling ergonomic chairs.\n\nIf Tredence has recently ventured into the production and sale of ergonomic chairs, I do not have specific information about that development. Ergonomic chairs are designed to provide comfort and support, promoting good posture and reducing the risk of musculoskeletal issues, which is increasingly important in today's work environments.\n\nFor the most accurate and up-to-date information, I recommend checking Tredence's official website or recent press releases to see if they have launched any new products, including ergonomic chairs."

In [19]:
prompt = f"""
Tell me about Ergonamic Toothbrush by Blue Data Consulting.
"""

generate_response(prompt)

'As of my last update in October 2023, there is no widely recognized product specifically called the "Ergonomic Toothbrush" by Blue Data Consulting. It\'s possible that this product is a recent development or a niche item that hasn\'t gained significant attention in mainstream markets.\n\nIf Blue Data Consulting has released an ergonomic toothbrush, it would likely focus on features designed to enhance user comfort and effectiveness in dental hygiene. Ergonomic toothbrushes typically have handles that are shaped to fit comfortably in the hand, reducing strain during brushing. They may also have specially designed bristles to improve cleaning efficiency while being gentle on the gums.\n\nFor the most accurate and up-to-date information, I recommend checking the official website of Blue Data Consulting or looking for recent product announcements or reviews. If you have any specific details or context about the product, feel free to share, and I can provide more tailored information!'

In [20]:
prompt = f"""
Tell me about Ultraslim Toothbrush by MorningStar.
"""

generate_response(prompt)

"The Ultraslim Toothbrush by MorningStar is designed to provide an effective and gentle cleaning experience for users. Its slim profile allows for easy maneuverability, making it easier to reach difficult areas in the mouth, such as between teeth and along the gum line. \n\nKey features often include:\n\n1. **Ultra-Slim Design**: The toothbrush is designed to be lightweight and easy to handle, which can be particularly beneficial for individuals with limited dexterity.\n\n2. **Soft Bristles**: Many models come with soft bristles that are gentle on the gums while still effectively removing plaque and debris.\n\n3. **Ergonomic Handle**: The handle is typically designed for comfort and ease of use, allowing for a better grip during brushing.\n\n4. **Compact Head**: A smaller brush head can help users access hard-to-reach areas more effectively.\n\n5. **Travel-Friendly**: Its slim design makes it a convenient option for travel, fitting easily into toiletry bags or travel cases.\n\n6. **Eco

In [21]:
prompt = f"""
Generate one line description for each of the following city:
1. Manila
2. Delhi
3. TredenceCity
"""

print(generate_response(prompt))

1. Manila: A vibrant metropolis blending rich history, diverse culture, and bustling urban life along the shores of Manila Bay.  
2. Delhi: A dynamic capital city of India, where ancient traditions meet modernity, showcasing a tapestry of cultures, cuisines, and historical landmarks.  
3. TredenceCity: A futuristic urban hub designed for innovation and sustainability, fostering a collaborative environment for technology and creativity.  


In [22]:
prompt = f"""
Generate one line description for each of the following city, check if the city exists, if it does not exist, say no description can be generated:
1. Manila
2. Delhi
3. MorningStarCity
"""

print(generate_response(prompt))

1. Manila: The vibrant capital of the Philippines, known for its rich history, bustling streets, and diverse culture.  
2. Delhi: The sprawling capital of India, blending ancient heritage with modern urban life, and serving as a political and cultural hub.  
3. MorningStarCity: No description can be generated.


## Conversational App

In [25]:
# creating a function to get outcome
def generate_response(messages,model='telcogpt'):
  response = client.chat.completions.create(
      model = 'myllm',
      messages = messages,
      temperature=0.5
  )
  return response.choices[0].message.content


In [26]:
messages = [{"role":"system","content":"You are a helpful assistant"},]
while True:
    uinput = input("You: ")
    print("You: ",uinput)
    if uinput.lower() in ['q','quit','exit']:
        break
    messages.append({"role":"user","content":uinput})
    response = generate_response(messages=messages)
    print(f"Bot: {response}")
    messages.append({"role":"assistant","content":response})


You:  Hi
Bot: Hello! How can I assist you today?
You:  tell me about the city Torontto
Bot: Toronto is the largest city in Canada and serves as the capital of the province of Ontario. It is a vibrant, multicultural metropolis known for its diverse population, economic significance, and cultural institutions. Here are some key points about Toronto:

### Geography
- **Location**: Toronto is situated in Southern Ontario, on the northwestern shore of Lake Ontario.
- **Neighborhoods**: The city is made up of various neighborhoods, each with its unique character, including Kensington Market, The Distillery District, and the Entertainment District.

### Demographics
- **Population**: Toronto has a population of over 2.8 million people, making it one of the most populous cities in North America.
- **Diversity**: The city is known for its cultural diversity, with over half of its residents being born outside of Canada. It is home to many ethnic communities, contributing to its rich cultural tap

In [ ]:
messages

In [28]:
def analyse_sentiment(text:str)->str:
    messages = [{"role":"system","content":"you are a sentiment analyzer model, for any given text return one word sentiment 'positive' or 'negative'"},]
    messages.append({'role':'user','content':text})
    response = generate_response(messages)
    return response

In [29]:
analyse_sentiment("I like this laptop, its awesome")

'positive'

In [30]:
analyse_sentiment("You killed it bro.")

'positive'

In [31]:
analyse_sentiment("You broke the bro code")

'negative'

## Thank You